## Задача
Необходимо создать модель, которая принимает на вход текст документа и наименование одного из двух пунктов, и возвращает возвращать соответствующий кусочек текста из текста документа (фрагмент, начало индекса фрагмента, конец индекса фрагмента)

Пример
```
    {'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], [1279], [1343] }
```

In [1]:
import json
from sklearn.utils import shuffle
import re
import numpy as np
import pandas as pd
import ast
import random
from tqdm import tqdm
from spacy.tokens import DocBin
from spacy.util import filter_spans
import spacy
from spacy.training.example import Example
from spacy.lang.ru import Russian
from sklearn.model_selection import train_test_split
from spacy.training import offsets_to_biluo_tags

In [13]:
with open('dataset\\train.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    data = json.load(f) #загнали все, что получилось в переменную
data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [2]:
import torch
torch.cuda.is_available()

True

### Сделаем предварительную обработку данных, для обучения модели Spacy

In [14]:
labels_ner = {'обеспечение исполнения контракта': 'EC', 'обеспечение гарантийных обязательств': 'PWO'}
def preprocessing(data):
    nlp = spacy.blank("ru")
    data_train = []
    data_missing = []
    for dict in data:
        text = dict.get('text')
        label = labels_ner.get(dict.get('label'))
        answer_start = dict.get('extracted_part').get('answer_start')[0]
        answer_end = dict.get('extracted_part').get('answer_end')[0]

        doc = nlp(dict.get('text'))
        entities = [(answer_start, answer_end, label)]
        tags = offsets_to_biluo_tags(doc, entities)

        if (answer_start == 0 and answer_end == 0) or ('-' in tags):
            data_missing.append((text, {'entities': []}))
        else:
            #print(text[answer_start-10:answer_end+10])
            row = (text, {'entities': [(answer_start, answer_end, label)]})
            data_train.append(row)
    return data_train, data_missing

data_train, data_missing = preprocessing(data)
#data_train, data_valid = train_test_split(data_train, random_state=12345, test_size=0.25)
#data_valid+=data_missing

'''
data_train, data_valid = train_test_split(preprocessing(data), random_state=12345, test_size=0.25)
print(len(data_train))
print(len(data_valid))
data_train[0]
'''

print(len(data_train))
#print(len(data_valid))
data_train[0]

d:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "РАЗДЕЛ 4. тельно); в) 1 процент цены контракта, ес..." with entities "[(1010, 1179, 'EC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "PAGE ПРОЕКТ КОНТРАКТА КОНТРАКТ № ______ на поставк..." with entities "[(1226, 1355, 'EC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ПРОЕКТ Государственный контракт №________ на поста..." with ent

1466


d:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Информационная карта о проведении закупки с единст..." with entities "[(1246, 1413, 'PWO')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "«УТВЕРЖДАЮ» ______________/___________/ «__» _____..." with entities "[(1253, 1412, 'PWO')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


('Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 

### Обучение модели Spacy

In [ ]:
spacy.require_gpu()
print(spacy.prefer_gpu())
nlp = spacy.blank("ru")

ner = nlp.add_pipe("ner") # spancat

ner.add_label("обеспечение исполнения контракта")
ner.add_label("обеспечение гарантийных обязательств")

optimizer = nlp.begin_training()


for epoch in range(5):
    random.shuffle(data_train)
    loss = {}
    for batch in spacy.util.minibatch(data_train, size=2):
        example = None
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)

        nlp.update([example], sgd=optimizer, losses=loss, drop=0.2)
    print('Epoch: ',epoch, ' losses: ', loss)

#nlp.to_disk("model")

In [3]:
nlp = spacy.load('model_best')

In [4]:
nlp.pipe_names

['ner']

In [5]:
analysis = nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component   Assigns          Requires   Scores          Retokenizes
-   ---------   --------------   --------   -------------   -----------
0   ner         doc.ents                    ents_f          False      
                token.ent_iob               ents_p                     
                token.ent_type              ents_r                     
                                            ents_per_type              

✔ No problems found.


In [8]:
def accuracy(true, pred):
    true_pred = 0
    for i in range(len(true)):
        if true[i] == pred[i]:
            true_pred+=1
    return true_pred/len(true)

In [17]:
def create_true_pred_arr(data, model):
    data_true = []
    data_pred = []

    lengh_max_true = 0
    count = 0
    for i in range(len(data)):
        if data[i][1].get('entities'):
            left = data[i][1].get('entities')[0][0]
            right = data[i][1].get('entities')[0][1]
            text = data[i][0][left:right]
            data_true.append(text)
            label = data[i][1].get('entities')[0][2]
        else:
            text = ''
            label = ''
            data_true.append('')


        doc = model(data[i][0])
        if list(doc.ents):
            #if len(list(doc.ents)) > 2:
            ents = [ent.text for ent in doc.ents if ent.label_ == label]
            if ents:
                data_pred.append(str(max(ents, key=len)))
            else:
                data_pred.append('')
            #data_pred.append(str(doc.ents[0]))
        else:
            data_pred.append('')
        count+=1

    return data_true, data_pred

#data_true, data_pred = create_true_pred_arr(data_valid, nlp)
#print(len(data_true))
#print(len(data_pred))

In [29]:
def create_true_pred_arr_2(data, model):
    data_true = []
    data_pred = []

    lengh_max_true = 0
    count = 0
    for i in range(len(data)):
        if data[i][1].get('entities'):
            left = data[i][1].get('entities')[0][0]
            right = data[i][1].get('entities')[0][1]
            text = data[i][0][left:right]
            data_true.append(text)
            label = data[i][1].get('entities')[0][2]
        else:
            text = ''
            label = ''
            data_true.append('')


        doc = model(data[i][0])
        if list(doc.ents):
            #if len(list(doc.ents)) > 2:
            ents = [ent.text for ent in doc.ents if ent.label_ == label]
            if ents:
                pred = str(max(ents, key=len))

            else:
                pred = ''
                
        if pred != text:
            print('Нужно: ', text)
            print('Получил: ', ents, end='\n\n')


    return data_true, data_pred

#data_true, data_pred = create_true_pred_arr(data_valid, nlp)
#print(len(data_true))
#print(len(data_pred))

In [30]:
data_true, data_pred = create_true_pred_arr_2(data_train, nlp)
#print('Метрика accuracy на тренировочной выборке:', accuracy(data_true, data_pred))

#data_true, data_pred = create_true_pred_arr_2(data_valid, nlp)
#print('Метрика accuracy на тестовой выборке:', accuracy(data_true, data_pred))

Нужно:  Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ___
Получил:  ['Участник закупки, с которым заключается контракт по результатам определения Поставщика в соответствии с п. 1 ч.1 ст.30 Закона № 44-ФЗ', 'Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт']

Нужно:  Размер обеспечения исполнения контракта (договора) составляет: 0,50% от цены
Получил:  ['Размер обеспечения исполнения контракта (договора) составляет: 0,50% от цены, по которой в соответствии с Законом № 44-ФЗ заключается контракт.']

Нужно:  Размер обеспечения исполнения контракта 5018.47 Российский рубль
Получил:  ['Размер обеспечения исполнения контракта 5018.47 Российский рубль', 'Размер обеспечения исполнения Контракта составляет 10 % от начальной (максимальной) цены контракта.']

Нужно:  Обеспечение исполнения настоящего Контракта установлено в разме

Исправленная метрика 0.7987721691678036
Базовая  0.7939972714870396

In [ ]:
#print(data_train[0][0])
doc = nlp(data_train[4][0])
print(doc.spans)

In [ ]:
data_train[0]

In [ ]:
try:
    s = re.search(str(max(list(doc.ents), key=len)), data[0][0])
    #print(s.span())
    #print(data[i][0][s.start():s.end()])
except Exception as e:
    pass

### Тестирование теорий

In [ ]:
text_test = data_train[0][0]

left = data_train[0][1].get('entities')[0]#[0]
right = data_train[0][1].get('entities')[0]#[1]
#print(text_test[left:right])
left

In [ ]:
doc = nlp(text_test)
print(doc.ents[0])
for ent in doc.ents:
    print(ent.label_)
    print(ent.text)

Такая задача называется извлечением информации (Information Extraction) или извлечением сущностей (Named Entity Recognition). В этой задаче используются метки, такие как имена собственные, даты, адреса, организации, и т.д., чтобы извлечь соответствующие куски текста. В области NLP существует также подзадача извлечения отношений между сущностями (Relation Extraction), где необходимо определить отношение между двумя или более сущностями в тексте.